# Movie Recommendation System


   Have you ever wondered how Netflix suggests movies to you based on the movies you have already watched? Or how does an e-commerce websites display options such as "Frequently Bought Together"? They may look relatively simple options but behind the scenes, a complex statistical algorithm executes in order to predict these recommendations. 
Such systems are called Recommender Systems, Recommendation Systems, or Recommendation Engines. A Recommender System is one of the most famous applications of Data science and Machine learning.

 A Recommender System employs a statistical algorithm that seeks to predict users' ratings for a particular entity, based on the similarity between the entities or similarity between the users that previously rated those entities. The intuition is that similar types of users are likely to have similar ratings for a set of entities.

Currently, many of the big tech companies out there use a Recommender System in one way or another. You can find them anywhere from Amazon (product recommendations) to YouTube (video recommendations) to Facebook (friend recommendations). The ability to recommend relevant products or services to users can be a huge boost for a company, which is why it's so common to find this technique employed in so many sites.

![Netflix movie recommendations](https://miro.medium.com/max/1600/1*dMR3xmufnmKiw4crlisQUA.png)
         
         

In this kernel we'll be building a  Movie Recommendation System using TMDB 5000 Movie Dataset. For beginners like me,this kernel will pretty much serve as a foundation in recommendation systems and will provide you with something to start with.

**Let's start.
**

There are basically three types of recommender systems:-

> * **Demographic filtering (Simple Recommender)** -  The System recommends the same movies to users with similar demographic features. They offer generalized recommendations to every user, based on movie popularity and/or genre. Since each user is different , this approach is considered to be too simple. The basic idea behind this system is that movies that are more popular and critically acclaimed will have a higher probability of being liked by the average audience.


> * **Content-Based Filtering** -  In content-based filtering, the similarity between different products is calculated on the basis of the attributes of the products. For instance, in a content-based movie recommender system, the similarity between the movies is calculated on the basis of genres, the actors in the movie, the director of the movie, etc. This system uses item metadata, such as genre, director, description, actors, etc. for movies, to make these recommendations. The general idea behind these recommender systems is that if a person liked a particular item, he or she will also like an item that is similar to it.

> * **Collaborative Filtering** -  The intuition behind collaborative filtering is that if a user A likes products X and Y, and if another user B likes product X, there is a fair bit of chance that he will like the product Y as well.  This system matches persons with similar interests and provides recommendations based on this matching. Collaborative filters do not require item metadata like its content-based counterparts.



## Data Visualization and Preprocessing

> The first step in every data science problem is to visualize and preprocess the data. We will do the same, so let's first import the TMBD data and see what it contains. 

In [ ]:
import pandas as pd    # Importing Pandas library
import numpy as np     # Importing Numpy library

#Importing data
df1 = pd.read_csv("../input/tmdb-movie-metadata/tmdb_5000_credits.csv") 
df2 = pd.read_csv("../input/tmdb-movie-metadata/tmdb_5000_movies.csv")


Let's see top 10 results of the dataset.

In [ ]:
df1.head(5)

The first dataset contains the following features:-
*     movie_id - A unique identifier for each movie.
*     title - Title of the movie
*     cast - The name of lead and supporting actors.
*     crew - The name of Director, Editor, Composer, Writer etc.


In [ ]:
df2.head(5)

The second dataset has the following features:-

*     budget - The budget in which the movie was made.
*     genre - The genre of the movie, Action, Comedy ,Thriller etc.
*     homepage - A link to the homepage of the movie.
*     id - This is infact the movie_id as in the first dataset.
*     keywords - The keywords or tags related to the movie.
*     original_language - The language in which the movie was made.
*     original_title - The title of the movie before translation or adaptation.
*     overview - A brief description of the movie.
*     popularity - A numeric quantity specifying the movie popularity.
*     production_companies - The production house of the movie.
*     production_countries - The country in which it was produced.
*     release_date - The date on which it was released.
*     revenue - The worldwide revenue generated by the movie.
*     runtime - The running time of the movie in minutes.
*     status - "Released" or "Rumored".
*     tagline - Movie's tagline.
*     title - Title of the movie.
*     vote_average - average ratings the movie recieved.
*     vote_count - the count of votes recieved.

Now,let's join the both datasets on 'id' coulmn.

In [ ]:
df1.columns = ['id','tittle','cast','crew']
df = df2.merge(df1,on='id')

**Let's see our final dataset.**

In [ ]:
df.head(5)

In [ ]:
print (df.columns)

> # **Demographic filtering**

 Before starting with demographic filtering:
* we need a metric to score or rate movie
* Calculate the score for every movie 
* Sort the scores and recommend the best rated movie to the users.

We can use the average ratings of the movie as the score but using this won't be fair enough since a movie with 8.9 average rating and only 3 votes cannot be considered better than the movie with 7.8 as as average rating but 40 votes. So,I will use IMDB's weighted rating formula to construct my chart. 

Formula for weighted rating:
![Weighted filtering](https://image.ibb.co/jYWZp9/wr.png)

 Where,<br> v = Number of votes for the movie. <br>
       m = minimum votes requires to listed in the chart. <br>
       R = Average rating of the movie. <br>
       c = Mean vote acroos the whole dataset.

In [ ]:
c = df['vote_average'].mean()
print (c)

>So, 6.092171559442011 is the average votes for a movie across the whole report.

>The next step is to determine an appropriate value for m, the minimum votes required to be listed in the chart. We will use 95th percentile as our cutoff. In other words, for a movie to feature in the charts, it must have more votes than at least 95% of the movies in the list.

In [ ]:
m = df['vote_count'].quantile(0.95)

We can filter out the movies that qualify for the chart


In [ ]:
qualified_movies = df.copy().loc[df['vote_count']>=m]

In [ ]:
qualified_movies.shape

> So there are 241 qualified movies in the list based on weighted rating.

Now we will define a funtion named ***weight_rating*** to calculate our metric for each qualified movie and define a new feature score, of which we'll calculate the value by applying this function to our DataFrame of qualified movies.

In [ ]:
def weighted_rating(x):
    v = x['vote_count']
    r = x['vote_average']
    return (v/(v+m) *r) + (m/(v+m) *c)

In [ ]:
qualified_movies['score'] = qualified_movies.apply(weighted_rating,axis=1)

  Finally sort the dataframe based on their score and output the title, vote count, vote average and weighted rating or score of the top 10 movies.

In [ ]:
#Sort movies based on score calculated above
qualified_movies = qualified_movies.sort_values('score', ascending=False)

#Print the top 10 movies
qualified_movies[['title','vote_count','vote_average','score']].head(10)

> These are the movies that have high weight rating score among the dataframe. We have made our first(though very basic) recommender. 
> Keep in mind  that these demographic recommender provide a general chart of recommended movies to all the users. They are not sensitive to the interests and tastes of a particular user. This is when we move on to a more refined system- Content Basesd Filtering.

# **Content-based filtering**

In this recommender system the description(plot) of the movie and metadata (cast,crew, genre and keywords) is used to find its similarity with other movies.
<img src="https://miro.medium.com/max/1026/1*BME1JjIlBEAI9BV5pOO5Mg.png" alt="Content based filtering" style="width:260px;"/>

>## Movie plot based recommender
 We will compute pairwise similarity scores for all movies based on their plot descriptions and recommend movies based on that similarity score. The plot description is given in the overview feature of our dataset. Let's take a look at the data. 

In [ ]:
df['overview'].head(5)

>Now we'll compute Term Frequency-Inverse Document Frequency (TF-IDF) vectors for each overview.

>Now if you are wondering what is term frequency , it is the relative frequency of a word in a document and is given as (term instances/total instances). Inverse Document Frequency is the relative count of documents containing the term is given as log(number of documents/documents with term) The overall importance of each word to the documents in which they appear is equal to TF * IDF.

>Fortunately, scikit-learn gives you a built-in TfIdfVectorizer class that produces the TF-IDF matrix in a couple of lines. 

In [ ]:
#Import TfIdfVectorizer from scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer

#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')

#Replace NaN with an empty string
df['overview'] = df['overview'].fillna('')

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(df['overview'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape


>We see that over 20,000 different words were used to describe the 4800 movies in our dataset.

>We got the matrix of vectors with plot. Now to compute a similarity score we use cosine similarity.There is no right answer to which score is the best. Different scores work well in different scenarios and it is often a good idea to experiment with different metrics.

<img src="https://neo4j.com/docs/graph-algorithms/current/images/cosine-similarity.png" alt="Cosine similarity" style="width:500px;"/>

>Since we have used the TF-IDF vectorizer, calculating the dot product will directly give us the cosine similarity score. Therefore, we will use sklearn's linear_kernel() instead of cosine_similarities() since it is faster.



In [ ]:
# Import linear_kernel from sklearn
from sklearn.metrics.pairwise import linear_kernel

#Compute cosine similarity using tfidf_matrix
cosine_sim = linear_kernel(tfidf_matrix , tfidf_matrix)



We are going to define a function that takes in a movie title as an input and outputs a list of the 10 most similar movies. Firstly, for this, we need a reverse mapping of movie titles and DataFrame indices. In other words, we need a mechanism to identify the index of a movie in our metadata DataFrame, given its title.

In [ ]:
#Construct a reverse map of indices and movie titles
indices = pd.Series(df.index, index=df['title']).drop_duplicates()

In [ ]:
# Function that takes in movie title as input and outputs most similar movies

def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return df2['title'].iloc[movie_indices]

In [ ]:
get_recommendations('The Dark Knight Rises')

 "The Dark Knight Rises" returns all Batman movies while it is more likely that the people who liked that movie are more inclined to enjoy other Christopher Nolan movies. This is something that cannot be captured by the present system.

> ## Metadata based recommender
> In this recommender system the metadata (cast,crew, genre and keywords) of the movie is used to find its similarity with other movies.

>From the cast, crew and keywords features, we need to extract the three most important actors, the director and the keywords associated with that movie. Right now, our data is present in the form of "stringified" lists , we need to convert it into a safe and usable structure

In [ ]:
# Parse the stringified features into their corresponding python objects
from ast import literal_eval

features = ['cast','crew','keywords','genres']

for i in features:
    df[i] = df[i].apply(literal_eval)




In [ ]:
# Get the director's name from the crew feature. If director is not listed, return NaN
def get_director(n):
    for i in n:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [ ]:
#Function for getting top 3 elements from the list
def get_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        #Check if more than 3 elements exist. If yes, return only first three. If no, return entire list.
        if len(names) > 3:
            names = names[:3]
        return names

    #Return empty list in case of missing/malformed data
    return []


In [ ]:
# Define new director, cast, genres and keywords features that are in a suitable form.
df['director'] = df['crew'].apply(get_director)
features = ['cast','keywords','genres']
for i in features:
    df[i] = df[i].apply(get_list)

In [ ]:
#Print the new features
df[['title', 'cast', 'director', 'keywords', 'genres']].head(3)

> Now we're going to clean the data converting them into lowercase and removing space between the names. Because our vectorizer counts the two person names as same if there first name is same.

In [ ]:
# Function to convert all strings to lower case and strip names of spaces
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        #Check if director exists. If not, return empty string
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''


In [ ]:
#Applying clean data function

features = ['cast','keywords','genres','director']

for i in features:
    df[i] = df[i].apply(clean_data)

> Now we're creating a metadata soup consists of all metadata like cast, keywords, genre etcc..,

In [ ]:
def create_soup(x):
    return ' '.join(x['keywords'])+' '+' '.join(x['cast'])+' '+' '.join(x['director'])+' '+' '.join(x['genres'])
df['soup'] = df.apply(create_soup,axis = 1)


> We'll use count vectorizer instead of TF-IDF here for recommendation. This is because we do not want to down-weight the presence of an actor/director if he or she has acted or directed in relatively more movies. It doesn't make much intuitive sense.

In [ ]:
# Import CountVectorizer and create the count matrix
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words = 'english')
cv_matrix = count.fit_transform(df['soup'])

In [ ]:
# Compute the Cosine Similarity matrix based on the count_matrix
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim2 = cosine_similarity(cv_matrix,cv_matrix)


In [ ]:
indices = pd.Series(df.index,index=df['title'])


We can now reuse our get_recommendations() function by passing in the new cosine_sim2 matrix as your second argument.

In [ ]:
#Now we can use get_recommendations with cosine_sim2 for metadata based recommender
get_recommendations('The Avengers',cosine_sim2)

We see that our recommender has been successful in capturing more information due to more metadata and has given us better recommendations.

# Collaborative filtering

> Our content based engine suffers from some severe limitations. It is only capable of suggesting movies which are close to a certain movie. That is, it is not capable of capturing tastes and providing recommendations across genres.

>Also, the engine that we built is not really personal in that it doesn't capture the personal tastes and biases of a user. Anyone querying our engine for recommendations based on a movie will receive the same recommendations for that movie, regardless of who she/he is.

<img src="https://miro.medium.com/max/1705/1*x8gTiprhLs7zflmEn1UjAQ.png" alt="Colloborative filtering" style="width:460px;"/>


>Therefore, in this section, we will use a technique called Collaborative Filtering to make recommendations to Movie Watchers. It is basically of two types:-
* **User based filtering** - These systems recommend products to a user that similar users have liked. For measuring the similarity between two users we can either use pearson correlation or cosine similarity. This filtering technique can be illustrated with an example. In the following matrixes, each row represents a user, while the columns correspond to different movies except the last one which records the similarity between that user and the target user. Each cell represents the rating that the user gives to that movie. Assume user E is the target. 

![User based filtering](https://cdn-images-1.medium.com/max/1000/1*9NBFo4AUQABKfoUOpE3F8Q.png)

>Since user A and F do not share any movie ratings in common with user E, their similarities with user E are not defined in Pearson Correlation. Therefore, we only need to consider user B, C, and D. Based on Pearson Correlation, we can compute the following similarity. 

![User based filtering](https://cdn-images-1.medium.com/max/1000/1*jZIMJzKM1hKTFftHfcSxRw.png)

From the above table we can see that user D is very different from user E as the Pearson Correlation between them is negative. He rated Me Before You higher than his rating average, while user E did the opposite. Now, we can start to fill in the blank for the movies that user E has not rated based on other users. 

![User based filtering](https://cdn-images-1.medium.com/max/1000/1*9TC6BrfxYttJwiATFAIFBg.png)

Although computing user-based CF is very simple, it suffers from several problems. One main issue is that users’ preference can change over time. It indicates that precomputing the matrix based on their neighboring users may lead to bad performance. To tackle this problem, we can apply item-based CF.

* **Item Based Collaborative Filtering** - Instead of measuring the similarity between users, the item-based CF recommends items based on their similarity with the items that the target user rated. Likewise, the similarity can be computed with Pearson Correlation or Cosine Similarity. The major difference is that, with item-based collaborative filtering, we fill in the blank vertically, as oppose to the horizontal manner that user-based CF does. The following table shows how to do so for the movie Me Before You. 

![Item based CF](https://cdn-images-1.medium.com/max/1000/1*LqFnWb-cm92HoMYBL840Ew.png)
It successfully avoids the problem posed by dynamic user preference as item-based CF is more static. However, several problems remain for this method. First, the main issue is scalability. The computation grows with both the customer and the product. The worst case complexity is O(mn) with m users and n items. In addition, sparsity is another concern. Take a look at the above table again. Although there is only one user that rated both Matrix and Titanic rated, the similarity between them is 1. In extreme cases, we can have millions of users and the similarity between two fairly different movies could be very high simply because they have similar rank for the only user who ranked them both.

I will not be implementing Collaborative Filtering from scratch. Instead, I will use the Surprise library that used extremely powerful algorithms like Singular Value Decomposition (SVD) to minimise RMSE (Root Mean Square Error) and give great recommendations.

In [ ]:
from surprise import SVD,Dataset,Reader,accuracy
from surprise.model_selection import cross_validate,KFold
reader = Reader()
ratings = pd.read_csv("../input/the-movies-dataset/ratings_small.csv")

In [ ]:
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
svd = SVD()
cross_validate(svd, data, measures=['RMSE', 'MAE'])

We get a mean Root Mean Sqaure Error of 0.895 approx which is more than good enough for our case. Let us now train on our dataset and arrive at predictions.

In [ ]:
trainset = data.build_full_trainset()
svd.fit(trainset)

Let us pick user with user Id 1 and check the ratings she/he has given.

In [ ]:
ratings[ratings['userId'] == 1]

In [ ]:
svd.predict(1, 302, 3)

For movie with ID 302, we get an estimated prediction of 2.776. One startling feature of this recommender system is that it doesn't care what the movie is (or what it contains). It works purely on the basis of an assigned movie ID and tries to predict ratings based on how the other users have predicted the movie.

## Conclusion

In this notebook, I have built 3 different recommendation engines based on different ideas and algorithms. They are as follows:

1. Demographical Recommender: This system used overall TMDB Vote Count and Vote Averages to build Top Movies Charts, in general and for a specific genre. The IMDB Weighted Rating System was used to calculate ratings on which the sorting was finally performed.
2. Content Based Recommender: We built two content based engines; one that took movie overview and taglines as input and the other which took metadata such as cast, crew, genre and keywords to come up with predictions. We also deviced a simple filter to give greater preference to movies with more votes and higher ratings.
3. Collaborative Filtering: We used the powerful Surprise Library to build a collaborative filter based on single value decomposition. The RMSE obtained was less than 1 and the engine gave estimated ratings for a given user and movie.

I would like to mention some excellent refereces that I learned from

1. https://www.kaggle.com/rounakbanik/movie-recommender-systems
2. https://www.kaggle.com/ibtesama/getting-started-with-a-movie-recommendation-system/
3. https://hackernoon.com/introduction-to-recommender-system-part-1-collaborative-filtering-singular-value-decomposition-44c9659c5e75

If you enjoyed reading the kernel , hit the ***upvote*** button! Please leave the feedback or suggestions below.
*Thank you !*